In [1]:
# data processing pseudocode

# read from data_algebraic directory
# parse the data, read exclusively from the support section
# copy 1 random support example and label it as the query example
# concatenate the support examples and the duplicate query example into a sentence
# tokenize each word into a number, pad it to a fixed length

# training pseudocode

# split into training and validation set -> already done
# feed the IN and OUT for each support example into the model
# feed only the IN for the query, have the model predict the OUT
# loss function to be determined ?

In [2]:
import random
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim



In [3]:
# for each file in the training set of 100000, parse the file

data = []

for i in range(1, 1001):
# for i in range(1, 100001):  
    file_index = f"{i:06d}"
    file_name = file_index + ".txt"
    file_path = f"data_algebraic/train/{file_name}"
    
    try:
        with open(file_path, 'r') as file:
            file_content = file.read()
            
            # Find the *SUPPORT* section
            start_index = file_content.find('*SUPPORT*')
            end_index = file_content.find('*QUERY*', start_index)
            
            # If both start and end indices are found
            if start_index != -1 and end_index != -1:
                # Extract the section and remove the *SUPPORT* label
                support_section = file_content[start_index:end_index]
                support_section = support_section.replace('*SUPPORT*', '', 1).strip()

                # Split by newlines, prepend 'EOS', and join lines
                lines = support_section.split('\n')
                lines = [f"EOS {line.strip()}" for line in lines if line.strip()]
                support_section_with_eos = ' '.join(lines)
                # Remove 'EOS ' from the start of the section if present
                if support_section_with_eos.startswith('EOS '):
                    support_section_with_eos = support_section_with_eos[4:]
                
                # Add the modified support section to the data list
                data.append(support_section_with_eos)
            else:
                print(f"No *SUPPORT* section found in file {file_path}.")

    except FileNotFoundError:
        print(f"File {file_path} not found.")

# # Print the parsed data for verification
# for item in data:
#     for in_part, out_part in item:
#         print(f"IN: {in_part} OUT: {out_part}")


In [4]:
data

['IN: dax OUT: BLUE EOS IN: lug fep blicket OUT: YELLOW PURPLE EOS IN: dax dax lug gazzer OUT: BLUE BLUE YELLOW BLUE BLUE YELLOW EOS IN: kiki fep blicket OUT: GREEN PURPLE EOS IN: lug blicket OUT: YELLOW PURPLE EOS IN: blicket OUT: PURPLE EOS IN: blicket lug dax fep kiki OUT: PURPLE YELLOW BLUE GREEN EOS IN: dax fep kiki OUT: BLUE GREEN EOS IN: blicket dax fep blicket OUT: PURPLE BLUE PURPLE EOS IN: lug fep dax OUT: YELLOW BLUE EOS IN: blicket fep blicket OUT: PURPLE PURPLE EOS IN: blicket fep kiki OUT: PURPLE GREEN EOS IN: lug OUT: YELLOW EOS IN: kiki fep kiki OUT: GREEN GREEN',
 'IN: lug zup OUT: PINK PINK EOS IN: dax OUT: PURPLE EOS IN: blicket fep wif blicket OUT: RED YELLOW YELLOW YELLOW EOS IN: blicket dax OUT: YELLOW PURPLE EOS IN: wif fep wif blicket OUT: RED YELLOW RED RED EOS IN: blicket OUT: YELLOW EOS IN: wif OUT: RED EOS IN: dax fep dax gazzer blicket zup OUT: YELLOW YELLOW YELLOW YELLOW PURPLE PURPLE PURPLE EOS IN: lug OUT: PINK EOS IN: wif gazzer dax OUT: PURPLE PURPLE R

In [5]:
data[100]

'IN: dax lug OUT: GREEN GREEN GREEN GREEN EOS IN: fep dax dax fep kiki zup gazzer OUT: PINK YELLOW GREEN GREEN YELLOW YELLOW EOS IN: wif dax kiki OUT: PURPLE GREEN GREEN EOS IN: dax dax lug OUT: GREEN GREEN GREEN GREEN GREEN EOS IN: fep dax fep kiki zup wif OUT: PURPLE YELLOW GREEN YELLOW YELLOW EOS IN: fep wif lug zup dax OUT: GREEN YELLOW PURPLE PURPLE PURPLE PURPLE EOS IN: dax OUT: GREEN EOS IN: gazzer OUT: PINK EOS IN: wif lug OUT: PURPLE PURPLE PURPLE PURPLE EOS IN: fep kiki OUT: YELLOW YELLOW EOS IN: dax zup dax OUT: GREEN GREEN EOS IN: wif kiki zup wif OUT: PURPLE PURPLE PURPLE EOS IN: wif zup gazzer OUT: PINK PURPLE EOS IN: wif kiki OUT: PURPLE PURPLE'

In [6]:
# tokenizer dictionary

input_list = ['SOS', 'dax', 'lug', 'wif', 'zup', 'fep', 'blicket', 'kiki', 'tufa', 'gazzer', 'EOS',]
output_list = ['SOS','RED', 'YELLOW', 'GREEN', 'BLUE', 'PURPLE', 'PINK', 'EOS']
vocab_list = ['SOS', 'dax', 'lug', 'wif', 'zup', 'fep', 'blicket', 'kiki', 'tufa', 'gazzer', 'RED', 'YELLOW', 'GREEN', 'BLUE', 'PURPLE', 'PINK',  'EOS',]

input_symbol_to_token = {symbol: float(idx + 1) for idx, symbol in enumerate(vocab_list)}
token_to_input_symbol = {float(idx + 1): symbol for idx, symbol in enumerate(vocab_list)}

# example

input_symbol = 'PINK'

input_token = input_symbol_to_token[input_symbol]

print(f"Input symbol '{input_symbol}' maps to token '{input_token}'")

# Reverse mapping example:
reverse_input_symbol = token_to_input_symbol[input_token]

print(f"Token '{input_token}' maps back to input symbol '{reverse_input_symbol}'")


Input symbol 'PINK' maps to token '16.0'
Token '16.0' maps back to input symbol 'PINK'


In [13]:
# tokenize the original data

def tokenize_sentence(sentence):
    tokens = sentence.split()
    print(tokens)
    token_ids = [input_symbol_to_token[token] for token in tokens]
    return token_ids

tokenized_data = []

data = data.split()
for sample in data:
    tokenized_sample = []
    for sentence in sample:
        tokens = tokenize_sentence(sentence)
        tokenized_sample.append((tokens))
    tokenized_data.append(tokenized_sample)

AttributeError: 'list' object has no attribute 'split'

In [ ]:
tokenized_data[100]

[([1.0, 3.0, 4.0, 2.0], [1.0, 14.0, 14.0, 14.0, 14.0, 2.0]),
 ([1.0, 7.0, 3.0, 3.0, 7.0, 9.0, 6.0, 11.0, 2.0],
  [1.0, 17.0, 13.0, 14.0, 14.0, 13.0, 13.0, 2.0]),
 ([1.0, 5.0, 3.0, 9.0, 2.0], [1.0, 16.0, 14.0, 14.0, 2.0]),
 ([1.0, 3.0, 3.0, 4.0, 2.0], [1.0, 14.0, 14.0, 14.0, 14.0, 14.0, 2.0]),
 ([1.0, 7.0, 3.0, 7.0, 9.0, 6.0, 5.0, 2.0],
  [1.0, 16.0, 13.0, 14.0, 13.0, 13.0, 2.0]),
 ([1.0, 7.0, 5.0, 4.0, 6.0, 3.0, 2.0],
  [1.0, 14.0, 13.0, 16.0, 16.0, 16.0, 16.0, 2.0]),
 ([1.0, 3.0, 2.0], [1.0, 14.0, 2.0]),
 ([1.0, 11.0, 2.0], [1.0, 17.0, 2.0]),
 ([1.0, 5.0, 4.0, 2.0], [1.0, 16.0, 16.0, 16.0, 16.0, 2.0]),
 ([1.0, 7.0, 9.0, 2.0], [1.0, 13.0, 13.0, 2.0]),
 ([1.0, 3.0, 6.0, 3.0, 2.0], [1.0, 14.0, 14.0, 2.0]),
 ([1.0, 5.0, 9.0, 6.0, 5.0, 2.0], [1.0, 16.0, 16.0, 16.0, 2.0]),
 ([1.0, 5.0, 6.0, 11.0, 2.0], [1.0, 17.0, 16.0, 2.0]),
 ([1.0, 5.0, 9.0, 2.0], [1.0, 16.0, 16.0, 2.0])]

In [ ]:
# Pad sequences to have uniform length
def pad_sequence(seq, max_len, padding_value=0):
    return seq + [padding_value] * (max_len - len(seq))

In [ ]:
# pick one random example to treat as the query 
query_data = []

for sample in tokenized_data:
    random_sample = random.choice(sample)
    # Add the selected tuple to query_data
    query_data.append(random_sample)

# query_data

In [ ]:
query_data[1]

([1.0, 5.0, 4.0, 11.0, 3.0, 2.0], [1.0, 12.0, 16.0, 16.0, 17.0, 2.0])

In [ ]:
tokenized_data[1]

[([1.0, 4.0, 6.0, 2.0], [1.0, 17.0, 17.0, 2.0]),
 ([1.0, 3.0, 2.0], [1.0, 16.0, 2.0]),
 ([1.0, 8.0, 7.0, 5.0, 8.0, 2.0], [1.0, 12.0, 13.0, 13.0, 13.0, 2.0]),
 ([1.0, 8.0, 3.0, 2.0], [1.0, 13.0, 16.0, 2.0]),
 ([1.0, 5.0, 7.0, 5.0, 8.0, 2.0], [1.0, 12.0, 13.0, 12.0, 12.0, 2.0]),
 ([1.0, 8.0, 2.0], [1.0, 13.0, 2.0]),
 ([1.0, 5.0, 2.0], [1.0, 12.0, 2.0]),
 ([1.0, 3.0, 7.0, 3.0, 11.0, 8.0, 6.0, 2.0],
  [1.0, 13.0, 13.0, 13.0, 13.0, 16.0, 16.0, 16.0, 2.0]),
 ([1.0, 4.0, 2.0], [1.0, 17.0, 2.0]),
 ([1.0, 5.0, 11.0, 3.0, 2.0], [1.0, 16.0, 16.0, 12.0, 2.0]),
 ([1.0, 5.0, 4.0, 11.0, 3.0, 2.0], [1.0, 12.0, 16.0, 16.0, 17.0, 2.0]),
 ([1.0, 5.0, 4.0, 11.0, 5.0, 6.0, 2.0],
  [1.0, 12.0, 12.0, 12.0, 12.0, 12.0, 17.0, 2.0]),
 ([1.0, 3.0, 6.0, 2.0], [1.0, 16.0, 16.0, 2.0]),
 ([1.0, 5.0, 7.0, 4.0, 11.0, 5.0, 6.0, 2.0],
  [1.0, 12.0, 12.0, 12.0, 12.0, 17.0, 12.0, 12.0, 2.0])]

In [ ]:
# Calculate the maximum input and output lengths across all sequences

max_input_len = max(len(inp) for sublist in tokenized_data for inp, _ in sublist)
max_output_len = max(len(out) for sublist in tokenized_data for _, out in sublist)

In [ ]:
class SequenceDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        # Return the number of sublists
        return len(self.data)

    def __getitem__(self, idx):
        
        # Get the list of tuples from the corresponding sublist
        index = self.data[idx]

        if isinstance(index, list):
            # tokenized_data is a list of a list of tuples
            # Pad each input-output pair in the sublist
            padded_data = [
                (pad_sequence(inp, max_input_len), pad_sequence(out, max_output_len))
                for inp, out in index
            ]
        else:
            # query_data is a list of tuples
            padded_data = [
                (pad_sequence(index[0], max_input_len), pad_sequence(index[1], max_output_len))
            ]

        
        # Convert each pair to a tensor
        padded_tensors = [(torch.tensor(input_padded), torch.tensor(output_padded)) for input_padded, output_padded in padded_data]

        return padded_tensors


In [ ]:
# Create datasets
train_dataset = SequenceDataset(tokenized_data)
query_dataset = SequenceDataset(query_data)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)
query_dataloader = DataLoader(query_dataset, batch_size=32, shuffle=False)

In [ ]:
query_dataset[0][0]


(tensor([1., 4., 7., 8., 2., 0., 0., 0., 0., 0.]),
 tensor([ 1., 13., 16.,  2.,  0.,  0.,  0.,  0.,  0.,  0.]))

In [ ]:
train_dataset[0]

[(tensor([1., 3., 2., 0., 0., 0., 0., 0., 0., 0.]),
  tensor([ 1., 15.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (tensor([1., 4., 7., 8., 2., 0., 0., 0., 0., 0.]),
  tensor([ 1., 13., 16.,  2.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (tensor([ 1.,  3.,  3.,  4., 11.,  2.,  0.,  0.,  0.,  0.]),
  tensor([ 1., 15., 15., 13., 15., 15., 13.,  2.,  0.,  0.])),
 (tensor([1., 9., 7., 8., 2., 0., 0., 0., 0., 0.]),
  tensor([ 1., 14., 16.,  2.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (tensor([1., 4., 8., 2., 0., 0., 0., 0., 0., 0.]),
  tensor([ 1., 13., 16.,  2.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (tensor([1., 8., 2., 0., 0., 0., 0., 0., 0., 0.]),
  tensor([ 1., 16.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (tensor([1., 8., 4., 3., 7., 9., 2., 0., 0., 0.]),
  tensor([ 1., 16., 13., 15., 14.,  2.,  0.,  0.,  0.,  0.])),
 (tensor([1., 3., 7., 9., 2., 0., 0., 0., 0., 0.]),
  tensor([ 1., 15., 14.,  2.,  0.,  0.,  0.,  0.,  0.,  0.])),
 (tensor([1., 8., 3., 7., 8., 2., 0., 0., 0., 0.]),
  tensor([ 1., 16.

In [ ]:
def decode_sequence(tokens, token_to_symbol):
    return [token_to_symbol[token] for token in tokens if token in token_to_symbol]

In [ ]:
# View a few examples from the train_dataset
for i in range(5):  # Print first 5 sublists/groups
    print(f"Group {i + 1}:")
    sublist = train_dataset[i]
    for j, (input_tensor, output_tensor) in enumerate(sublist):
        input_tokens = input_tensor.tolist()
        output_tokens = output_tensor.tolist()
        
        print(f" Example {j + 1}:")
        print(f"  Input tokens: {input_tokens}")
        print(f"  Output tokens: {output_tokens}")

        # Decode tokens to symbols for readability
        input_symbols = decode_sequence(input_tokens, token_to_input_symbol)
        output_symbols = decode_sequence(output_tokens, token_to_input_symbol)

        print(f"  Decoded Input: {input_symbols}")
        print(f"  Decoded Output: {output_symbols}")
    print("=" * 50)

Group 1:
 Example 1:
  Input tokens: [1.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'dax', 'EOS']
  Decoded Output: ['SOS', 'BLUE', 'EOS']
 Example 2:
  Input tokens: [1.0, 4.0, 7.0, 8.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 13.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'lug', 'fep', 'blicket', 'EOS']
  Decoded Output: ['SOS', 'YELLOW', 'PURPLE', 'EOS']
 Example 3:
  Input tokens: [1.0, 3.0, 3.0, 4.0, 11.0, 2.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 15.0, 15.0, 13.0, 15.0, 15.0, 13.0, 2.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'dax', 'dax', 'lug', 'gazzer', 'EOS']
  Decoded Output: ['SOS', 'BLUE', 'BLUE', 'YELLOW', 'BLUE', 'BLUE', 'YELLOW', 'EOS']
 Example 4:
  Input tokens: [1.0, 9.0, 7.0, 8.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 14.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'kiki', 'fep', 'blicket', 'E

In [ ]:
# View a few examples from the query dataset
for i in range(5):  # Print first 5 tuples
    print(f"Group {i + 1}:")
    input_tensor, output_tensor = query_dataset[i][0]
    input_tokens = input_tensor.tolist()
    output_tokens = output_tensor.tolist()
        
    print(f"  Input tokens: {input_tokens}")
    print(f"  Output tokens: {output_tokens}")

    # Decode tokens to symbols for readability
    input_symbols = decode_sequence(input_tokens, token_to_input_symbol)
    output_symbols = decode_sequence(output_tokens, token_to_input_symbol)

    print(f"  Decoded Input: {input_symbols}")
    print(f"  Decoded Output: {output_symbols}")
    print("=" * 50)

Group 1:
  Input tokens: [1.0, 4.0, 7.0, 8.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 13.0, 16.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'lug', 'fep', 'blicket', 'EOS']
  Decoded Output: ['SOS', 'YELLOW', 'PURPLE', 'EOS']
Group 2:
  Input tokens: [1.0, 5.0, 4.0, 11.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 12.0, 16.0, 16.0, 17.0, 2.0, 0.0, 0.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'wif', 'lug', 'gazzer', 'dax', 'EOS']
  Decoded Output: ['SOS', 'RED', 'PURPLE', 'PURPLE', 'PINK', 'EOS']
Group 3:
  Input tokens: [1.0, 9.0, 3.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 15.0, 15.0, 15.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'kiki', 'dax', 'EOS']
  Decoded Output: ['SOS', 'BLUE', 'BLUE', 'BLUE', 'BLUE', 'EOS']
Group 4:
  Input tokens: [1.0, 3.0, 4.0, 5.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  Output tokens: [1.0, 13.0, 13.0, 15.0, 15.0, 2.0, 0.0, 0.0, 0.0, 0.0]
  Decoded Input: ['SOS', 'dax', 'lug', 'wif', 'EOS']
  D

In [ ]:
# output dimensions -> number of tokens 

class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # Forward pass through LSTM
        out, _ = self.lstm(x)
        # Output layer
        out = self.fc(out[:, -1, :])  # Take the output from the last time step
        return out

# what is the input size in a seq2seq?

# Hyperparameters
input_size = len(input_list)
hidden_size = 50
output_size = len(output_list)
num_layers = 1
learning_rate = 0.001
num_epochs = 100

In [ ]:
output_size

8

In [ ]:
# Create the model
model = LSTM(input_size, hidden_size, output_size, num_layers)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
import numpy as np
for epoch in range (num_epochs):
    model.train()
    for batch_idx, batch in enumerate(train_dataloader):
        # Each batch is a list of tuples, so you need to extract inputs and targets
        batch_loss = 0.0  # Track the loss for each batch
        batch = np.array(batch)
        input_batch, target_batch = np.swapaxes(batch, 0, 1)
        print(input_batch.shape)
        # for seq, target in batch:
        #     # Ensure seq and target are float tensors
        #     seq = seq.float()
        #     if seq.dim() == 2:
        #         seq = seq.unsqueeze(0)
                
        #     target = target.float()
        #     if (epoch == 0 and batch_idx == 0):
        #         print(seq.shape)
        #         # print(seq)

(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 8, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 8, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(14, 32, 10)
(

In [ ]:
# Training loop using DataLoader
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for batch in train_dataloader:
        # Each batch is a list of tuples, so you need to extract inputs and targets
        batch_loss = 0.0  # Track the loss for each batch
        for seq, target in batch:
            # Ensure seq and target are float tensors
            seq = seq.float()
            target = target.float()

            # Add batch dimension if needed
            if seq.dim() == 2:  # if not already batched
                seq = seq.unsqueeze(0)

            # Forward pass
            optimizer.zero_grad()
            output = model(seq)

            # Compute loss
            loss = criterion(output, target)
            batch_loss += loss.item()  # Accumulate the loss for each sample

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {batch_loss:.4f}')

# Test with a new sequence using the query_dataloader
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    for test_batch in query_dataloader:
        for test_seq, _ in test_batch:  # We are only interested in inputs here
            # Ensure test_seq is a float tensor
            test_seq = test_seq.float()

            # Add batch dimension if needed
            if test_seq.dim() == 2:  # if not already batched
                test_seq = test_seq.unsqueeze(0)

            predicted_output = model(test_seq)
            print(f'Test sequence: {test_seq.squeeze().tolist()}')
            print(f'Predicted next value: {predicted_output.squeeze().item()}')


RuntimeError: input.size(-1) must be equal to input_size. Expected 11, got 10